In [2]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import datetime
import gradio as gr
import sklearn

import pickle

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose



In [3]:
IMPORTANT_LMS1 = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

headersbc = ["label"] # Label column

for lm in IMPORTANT_LMS1:
    headersbc += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]


def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in IMPORTANT_LMS1:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [4]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame from OpenCV to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)


def save_frame_as_image(frame, message: str = None):
    '''
    Save a frame as image to display the error
    '''
    now = datetime.datetime.now()

    if message:
        cv2.putText(frame, message, (50, 150), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
        
    print("Saving ...")
    cv2.imwrite(f"C:/SRM/sem5/ml/project/Exercise-Correction/core/bicep_model/bicep_{now}.jpg", frame)


def calculate_angle(point1: list, point2: list, point3: list) -> float:
    '''
    Calculate the angle between 3 points
    Unit of the angle will be in Degree
    '''
    point1 = np.array(point1)
    point2 = np.array(point2)
    point3 = np.array(point3)

    # Calculate algo
    angleInRad = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angleInDeg = np.abs(angleInRad * 180.0 / np.pi)

    angleInDeg = angleInDeg if angleInDeg <= 180 else 360 - angleInDeg
    return angleInDeg


def extract_important_keypoints(results, IMPORTANT_LMS1: list) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in IMPORTANT_LMS1:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


In [5]:
class BicepPoseAnalysis:
    def __init__(self, side: str, stage_down_threshold: float, stage_up_threshold: float, peak_contraction_threshold: float, loose_upper_arm_angle_threshold: float, visibility_threshold: float):
        # Initialize thresholds
        self.stage_down_threshold = stage_down_threshold
        self.stage_up_threshold = stage_up_threshold
        self.peak_contraction_threshold = peak_contraction_threshold
        self.loose_upper_arm_angle_threshold = loose_upper_arm_angle_threshold
        self.visibility_threshold = visibility_threshold

        self.side = side
        self.counter = 0
        self.stage = "down"
        self.is_visible = True
        self.detected_errors = {
            "LOOSE_UPPER_ARM": 0,
            "PEAK_CONTRACTION": 0,
        }

        # Params for loose upper arm error detection
        self.loose_upper_arm = False

        # Params for peak contraction error detection
        self.peak_contraction_angle = 1000
        self.peak_contraction_frame = None
    
    def get_joints(self, landmarks) -> bool:
        '''
        Check for joints' visibility then get joints coordinate
        '''
        side = self.side.upper()

        # Check visibility
        joints_visibility = [ landmarks[mp_pose.PoseLandmark[f"{side}_SHOULDER"].value].visibility, landmarks[mp_pose.PoseLandmark[f"{side}_ELBOW"].value].visibility, landmarks[mp_pose.PoseLandmark[f"{side}_WRIST"].value].visibility ]

        is_visible = all([ vis > self.visibility_threshold for vis in joints_visibility ])
        self.is_visible = is_visible

        if not is_visible:
            return self.is_visible
        
        # Get joints' coordinates
        self.shoulder = [ landmarks[mp_pose.PoseLandmark[f"{side}_SHOULDER"].value].x, landmarks[mp_pose.PoseLandmark[f"{side}_SHOULDER"].value].y ]
        self.elbow = [ landmarks[mp_pose.PoseLandmark[f"{side}_ELBOW"].value].x, landmarks[mp_pose.PoseLandmark[f"{side}_ELBOW"].value].y ]
        self.wrist = [ landmarks[mp_pose.PoseLandmark[f"{side}_WRIST"].value].x, landmarks[mp_pose.PoseLandmark[f"{side}_WRIST"].value].y ]

        return self.is_visible
    
    def analyze_pose(self, landmarks, frame):
        '''
        - Bicep Counter
        - Errors Detection
        '''
        self.get_joints(landmarks)

        # Cancel calculation if visibility is poor
        if not self.is_visible:
            return (None, None)

        # * Calculate curl angle for counter
        bicep_curl_angle = int(calculate_angle(self.shoulder, self.elbow, self.wrist))
        if bicep_curl_angle > self.stage_down_threshold:
            self.stage = "down"
        elif bicep_curl_angle < self.stage_up_threshold and self.stage == "down":
            self.stage = "up"
            self.counter += 1
        
        # * Calculate the angle between the upper arm (shoulder & joint) and the Y axis
        shoulder_projection = [ self.shoulder[0], 1 ] # Represent the projection of the shoulder to the X axis
        ground_upper_arm_angle = int(calculate_angle(self.elbow, self.shoulder, shoulder_projection))

        # * Evaluation for LOOSE UPPER ARM error
        if ground_upper_arm_angle > self.loose_upper_arm_angle_threshold:
            # Limit the saved frame
            if not self.loose_upper_arm:
                self.loose_upper_arm = True
                # save_frame_as_image(frame, f"Loose upper arm: {ground_upper_arm_angle}")
                self.detected_errors["LOOSE_UPPER_ARM"] += 1
        else:
            self.loose_upper_arm = False
        
        # * Evaluate PEAK CONTRACTION error
        if self.stage == "up" and bicep_curl_angle < self.peak_contraction_angle:
            # Save peaked contraction every rep
            self.peak_contraction_angle = bicep_curl_angle
            self.peak_contraction_frame = frame
            
        elif self.stage == "down":
            # * Evaluate if the peak is higher than the threshold if True, marked as an error then saved that frame
            if self.peak_contraction_angle != 1000 and self.peak_contraction_angle >= self.peak_contraction_threshold:
                # save_frame_as_image(self.peak_contraction_frame, f"{self.side} - Peak Contraction: {self.peak_contraction_angle}")
                self.detected_errors["PEAK_CONTRACTION"] += 1
            
            # Reset params
            self.peak_contraction_angle = 1000
            self.peak_contraction_frame = None
        
        return (bicep_curl_angle, ground_upper_arm_angle)

In [6]:
imp_lnd = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

headersquat = ["label"] # Label column

for lm in imp_lnd:
    headersquat += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]


def extract_important_keypoints_sq(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmark = results.pose_landmarks.landmark

    data = []
    for lm in imp_lnd:
        keypoint = landmark[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [8]:
import math 


def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def analyze_foot_knee_placement(results, stage: str, foot_shoulder_ratio_thresholds: list, knee_foot_ratio_thresholds: dict, visibility_threshold: int) -> dict:
    '''
    Calculate the ratio between the foot and shoulder for FOOT PLACEMENT analysis
    
    Calculate the ratio between the knee and foot for KNEE PLACEMENT analysis

    Return result explanation:
        -1: Unknown result due to poor visibility
        0: Correct knee placement
        1: Placement too tight
        2: Placement too wide
    '''
    analyzed_results = {
        "foot_placement": -1,
        "knee_placement": -1,
    }

    landmark = results.pose_landmarks.landmark

    # * Visibility check of important landmark for foot placement analysis
    left_foot_index_vis = landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].visibility
    right_foot_index_vis = landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].visibility

    left_knee_vis = landmark[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
    right_knee_vis = landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility

    # If visibility of any keypoints is low cancel the analysis
    if (left_foot_index_vis < visibility_threshold or right_foot_index_vis < visibility_threshold or left_knee_vis < visibility_threshold or right_knee_vis < visibility_threshold):
        return analyzed_results
    
    # * Calculate shoulder width
    left_shoulder = [landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    shoulder_width = calculate_distance(left_shoulder, right_shoulder)

    # * Calculate 2-foot width
    left_foot_index = [landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    right_foot_index = [landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
    foot_width = calculate_distance(left_foot_index, right_foot_index)

    # * Calculate foot and shoulder ratio
    foot_shoulder_ratio = round(foot_width / shoulder_width, 1)

    # * Analyze FOOT PLACEMENT
    min_ratio_foot_shoulder, max_ratio_foot_shoulder = foot_shoulder_ratio_thresholds
    if min_ratio_foot_shoulder <= foot_shoulder_ratio <= max_ratio_foot_shoulder:
        analyzed_results["foot_placement"] = 0
    elif foot_shoulder_ratio < min_ratio_foot_shoulder:
        analyzed_results["foot_placement"] = 1
    elif foot_shoulder_ratio > max_ratio_foot_shoulder:
        analyzed_results["foot_placement"] = 2
    
    # * Visibility check of important landmark for knee placement analysis
    left_knee_vis = landmark[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
    right_knee_vis = landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility

    # If visibility of any keypoints is low cancel the analysis
    if (left_knee_vis < visibility_threshold or right_knee_vis < visibility_threshold):
        print("Cannot see foot")
        return analyzed_results

    # * Calculate 2 knee width
    left_knee = [landmark[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmark[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    right_knee = [landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    knee_width = calculate_distance(left_knee, right_knee)

    # * Calculate foot and shoulder ratio
    knee_foot_ratio = round(knee_width / foot_width, 1)

    # * Analyze KNEE placement
    up_min_ratio_knee_foot, up_max_ratio_knee_foot = knee_foot_ratio_thresholds.get("up")
    middle_min_ratio_knee_foot, middle_max_ratio_knee_foot = knee_foot_ratio_thresholds.get("middle")
    down_min_ratio_knee_foot, down_max_ratio_knee_foot = knee_foot_ratio_thresholds.get("down")

    if stage == "up":
        if up_min_ratio_knee_foot <= knee_foot_ratio <= up_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 0
        elif knee_foot_ratio < up_min_ratio_knee_foot:
            analyzed_results["knee_placement"] = 1
        elif knee_foot_ratio > up_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 2
    elif stage == "middle":
        if middle_min_ratio_knee_foot <= knee_foot_ratio <= middle_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 0
        elif knee_foot_ratio < middle_min_ratio_knee_foot:
            analyzed_results["knee_placement"] = 1
        elif knee_foot_ratio > middle_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 2
    elif stage == "down":
        if down_min_ratio_knee_foot <= knee_foot_ratio <= down_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 0
        elif knee_foot_ratio < down_min_ratio_knee_foot:
            analyzed_results["knee_placement"] = 1
        elif knee_foot_ratio > down_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 2
    
    return analyzed_results



In [9]:
with open("C:/SRM/sem5/ml/project/ml proj/squat_model/model/LR_model.pkl", "rb") as f:
    count_model = pickle.load(f)

In [10]:
imp_pu = [
    "NOSE",
    "LEFT_EYE_INNER",
    "LEFT_EYE",
    "LEFT_EYE_OUTER",
    "RIGHT_EYE_INNER",
    "RIGHT_EYE",
    "RIGHT_EYE_OUTER",
    "LEFT_EAR",
    "RIGHT_EAR",
    "MOUTH_LEFT",
    "MOUTH_RIGHT",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_PINKY",
    "RIGHT_PINKY",
    "LEFT_INDEX",
    "RIGHT_INDEX",
    "LEFT_THUMB",
    "RIGHT_THUMB",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX"
]
headerspu = ["label"]
for lm in imp_pu:
    headerspu += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [11]:
def extract_important_keypoints_pu(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in imp_pu:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [12]:
# Load model
with open("C:/SRM/sem5/ml/project/ml proj/pushup_model/pickles/RF_model.pkl", "rb") as f:
    sklearn_model_pu = pickle.load(f)

# Load input scaler
with open("C:/SRM/sem5/ml/project/ml proj/pushup_model/pickles/input_scaler.pkl", "rb") as f2:
    input_scaler_pu = pickle.load(f2)

# Transform prediction into class
def get_class(prediction: float) -> str:
    return {
        0: "C",
        1: "H",
        2: "L",
    }.get(prediction)

In [14]:
# Load model
with open("C:/SRM/sem5/ml/project/ml proj/pushup_model/pickles/pushup_dp.pkl", "rb") as f:
    DL_model_pu = pickle.load(f)

In [15]:
# Determine important landmarks for plank
IMP_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

headersplank = ["label"] # Label column

for lm in IMP_LMS:
    headersplank += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [16]:
def extract_important_keypoints_pl(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in IMP_LMS:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [17]:
# Load model
with open("C:/SRM/sem5/ml/project/ml proj/plank_model/model/LR_model.pkl", "rb") as f:
    sklearn_model_pl = pickle.load(f)

# Load input scaler
with open("C:/SRM/sem5/ml/project/ml proj/plank_model/model/input_scaler.pkl", "rb") as f2:
    input_scaler_pl = pickle.load(f2)

# Transform prediction into class
def get_class_plank(prediction: float) -> str:
    return {
        0: "C",
        1: "H",
        2: "L",
    }.get(prediction)


In [22]:

# Initialize MediaPipe pose and drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Define constants for Bicep analysis
VISIBILITY_THRESHOLD_BICEP = 0.65
STAGE_UP_THRESHOLD_BICEP = 90
STAGE_DOWN_THRESHOLD_BICEP = 120
PEAK_CONTRACTION_THRESHOLD_BICEP = 60
LOOSE_UPPER_ARM_ANGLE_THRESHOLD_BICEP = 40

# Bicep Pose Analysis class initialization (you should have this class defined)
left_arm_analysis = BicepPoseAnalysis(side="left", stage_down_threshold=STAGE_DOWN_THRESHOLD_BICEP,
                                       stage_up_threshold=STAGE_UP_THRESHOLD_BICEP,
                                       peak_contraction_threshold=PEAK_CONTRACTION_THRESHOLD_BICEP,
                                       loose_upper_arm_angle_threshold=LOOSE_UPPER_ARM_ANGLE_THRESHOLD_BICEP,
                                       visibility_threshold=VISIBILITY_THRESHOLD_BICEP)

right_arm_analysis = BicepPoseAnalysis(side="right", stage_down_threshold=STAGE_DOWN_THRESHOLD_BICEP,
                                        stage_up_threshold=STAGE_UP_THRESHOLD_BICEP,
                                        peak_contraction_threshold=PEAK_CONTRACTION_THRESHOLD_BICEP,
                                        loose_upper_arm_angle_threshold=LOOSE_UPPER_ARM_ANGLE_THRESHOLD_BICEP,
                                        visibility_threshold=VISIBILITY_THRESHOLD_BICEP)

# Define the analyze_pose function for Bicep Curl Analysis
def analyze_bicep_pose(video):
    cap = cv2.VideoCapture(video)
    results_summary = []

    with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                (left_bicep_curl_angle, _) = left_arm_analysis.analyze_pose(landmarks=landmarks, frame=image)
                (right_bicep_curl_angle, _) = right_arm_analysis.analyze_pose(landmarks=landmarks, frame=image)

                left_bicep_curl_angle = left_bicep_curl_angle if left_bicep_curl_angle is not None else 0.0
                right_bicep_curl_angle = right_bicep_curl_angle if right_bicep_curl_angle is not None else 0.0

                result_summary = (
                    f"Left Angle: {left_bicep_curl_angle:.2f}, Right Angle: {right_bicep_curl_angle:.2f}, "
                    f"Left Counter: {left_arm_analysis.counter}, Right Counter: {right_arm_analysis.counter}"
                )
                results_summary.append(result_summary)

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imshow("Bicep Pose Analysis", image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

    summary_text = "\n".join(results_summary)
    return summary_text

# Define constants for Squat analysis
PREDICTION_PROB_THRESHOLD_SQUAT = 0.7
VISIBILITY_THRESHOLD_SQUAT = 0.6
FOOT_SHOULDER_RATIO_THRESHOLDS = [1.2, 2.8]
KNEE_FOOT_RATIO_THRESHOLDS = {
    "up": [0.5, 1.0],
    "middle": [0.7, 1.0],
    "down": [0.7, 1.1],
}

# Define the process_video function for Squat Analysis
def analyze_squat_pose(video):
    cap = cv2.VideoCapture(video)
    counter = 0
    current_stage = ""
    results_summary = []

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            if not results.pose_landmarks:
                continue

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            try:
                row = extract_important_keypoints_sq(results)
                X = pd.DataFrame([row], columns=headersquat[1:])

                predicted_class = count_model.predict(X)[0]
                predicted_class = "down" if predicted_class == 0 else "up"
                prediction_probabilities = count_model.predict_proba(X)[0]
                prediction_probability = round(prediction_probabilities[prediction_probabilities.argmax()], 2)

                if predicted_class == "down" and prediction_probability >= PREDICTION_PROB_THRESHOLD_SQUAT:
                    current_stage = "down"
                elif current_stage == "down" and predicted_class == "up" and prediction_probability >= PREDICTION_PROB_THRESHOLD_SQUAT: 
                    current_stage = "up"
                    counter += 1

                analyzed_results = analyze_foot_knee_placement(results=results, stage=current_stage, 
                                                                foot_shoulder_ratio_thresholds=FOOT_SHOULDER_RATIO_THRESHOLDS, 
                                                                knee_foot_ratio_thresholds=KNEE_FOOT_RATIO_THRESHOLDS, 
                                                                visibility_threshold=VISIBILITY_THRESHOLD_SQUAT)

                foot_placement_evaluation = analyzed_results["foot_placement"]
                knee_placement_evaluation = analyzed_results["knee_placement"]
                
                foot_placement = ["Correct", "Too tight", "Too wide", "UNK"][foot_placement_evaluation] if foot_placement_evaluation != -1 else "UNK"
                knee_placement = ["Correct", "Too tight", "Too wide", "UNK"][knee_placement_evaluation] if knee_placement_evaluation != -1 else "UNK"

                results_summary.append({
                    "counter": counter,
                    "stage": current_stage,
                    "prediction_probability": prediction_probability,
                    "foot_placement": foot_placement,
                    "knee_placement": knee_placement
                })

            except Exception as e:
                print(f"Error: {e}")

        cap.release()
    
    summary_text = "\n".join([f"Probability: {result['prediction_probability']}, "
                              f"Foot Placement: {result['foot_placement']}, Knee Placement: {result['knee_placement']}" 
                              for result in results_summary])
    
    return summary_text

#pushup analysis
PREDICTION_PROB_THRESHOLD_PUSHUP=0.6
def analyze_pushup_pose(video):
    cap = cv2.VideoCapture(video)  
    current_stage = ""
    results_summary = []

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()

            if not ret:
                break

            # Recolor image from BGR to RGB for mediapipe
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            if not results.pose_landmarks:
                results_summary.append("No human found")
                continue

            try:
                row = extract_important_keypoints_pu(results)
                X = pd.DataFrame([row], columns=headerspu[1:])
                X = pd.DataFrame(input_scaler_pu.transform(X))

                # Make prediction
                predictions_pu = DL_model_pu.predict(X)
                predicted_class_pu = np.argmax(predictions_pu)  # Get the index of the highest predicted probability
                prediction_probability_pu = np.max(predictions_pu)  # Get the maximum probability

                if predicted_class_pu == 0 and prediction_probability_pu >= PREDICTION_PROB_THRESHOLD_PUSHUP:
                    current_stage_pu = "Correct"
                elif predicted_class_pu == 2 and prediction_probability_pu >= PREDICTION_PROB_THRESHOLD_PUSHUP:
                    current_stage_pu = "Low back"
                elif predicted_class_pu == 1 and prediction_probability_pu >= PREDICTION_PROB_THRESHOLD_PUSHUP:
                    current_stage_pu = "High back"
                else:
                    current_stage_pu = "Unknown"

                results_summary.append(f"Stage: {current_stage_pu}, Probability: {prediction_probability_pu:.2f}")

            except Exception as e:
                results_summary.append(f"Error: {e}")

        cap.release()

    return "\n".join(results_summary)

#PLANK ANALYSIS
PREDICTION_PROB_THRESHOLD_PLANK=0.6
def analyze_plank_pose(video):
    cap = cv2.VideoCapture(video)
    prediction_probability_threshold = PREDICTION_PROB_THRESHOLD_PLANK
    output_text = []

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()

            if not ret:
                break

            # Convert BGR to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Process the frame with MediaPipe
            results = pose.process(image)

            if not results.pose_landmarks:
                output_text.append("No human found")
                continue

            try:
                # Extract keypoints and make predictions
                row = extract_important_keypoints_pl(results)
                X = pd.DataFrame([row], columns=headersplank[1:])
                X = pd.DataFrame(input_scaler_pl.transform(X))

                # Prediction
                predicted_class = sklearn_model_pl.predict(X)[0]
                predicted_class = get_class(predicted_class)
                prediction_probability = sklearn_model_pl.predict_proba(X)[0]
                prob_value = round(prediction_probability[np.argmax(prediction_probability)], 2)

                if predicted_class == "C" and prediction_probability[prediction_probability.argmax()] >= prediction_probability_threshold:
                    current_stage = "Correct"
                elif predicted_class == "L" and prediction_probability[prediction_probability.argmax()] >= prediction_probability_threshold: 
                    current_stage = "Low back"
                elif predicted_class == "H" and prediction_probability[prediction_probability.argmax()] >= prediction_probability_threshold: 
                    current_stage = "High back"
                else:
                    current_stage = "Unknown"

                # Append result
                output_text.append(f"Stage: {current_stage}, Probability: {prob_value}")

            except Exception as e:
                output_text.append(f"Error: {e}")
        
        cap.release()

    return "\n".join(output_text)


# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Bicep, Squat, Pushup, and Plank Pose Analysis")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("## Bicep Curl Analysis")
            bicep_input = gr.Video(label="Upload a Video for Bicep Curl Analysis")
            bicep_output = gr.Textbox(label="Bicep Analysis Results", interactive=False)
            bicep_button = gr.Button("Analyze Bicep Curl")
            bicep_button.click(analyze_bicep_pose, inputs=bicep_input, outputs=bicep_output)

        with gr.Column():
            gr.Markdown("## Squat Analysis")
            squat_input = gr.Video(label="Upload a Video for Squat Analysis")
            squat_output = gr.Textbox(label="Squat Analysis Results", interactive=False)
            squat_button = gr.Button("Analyze Squat")
            squat_button.click(analyze_squat_pose, inputs=squat_input, outputs=squat_output)

        with gr.Column():
            gr.Markdown("## Pushup Analysis")
            pushup_input = gr.Video(label="Upload a Video for Pushup Analysis")
            pushup_output = gr.Textbox(label="Pushup Analysis Results", interactive=False)
            pushup_button = gr.Button("Analyze Pushup")
            pushup_button.click(analyze_pushup_pose, inputs=pushup_input, outputs=pushup_output)

        with gr.Column():
            gr.Markdown("## Plank Analysis")
            plank_input = gr.Video(label="Upload a Video for Plank Analysis")
            plank_output = gr.Textbox(label="Plank Analysis Results", interactive=False)
            plank_button = gr.Button("Analyze Plank")
            plank_button.click(analyze_plank_pose, inputs=plank_input, outputs=plank_output)

# Launch Gradio app
demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
